In [1]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import math
import os
import pandas as pd

# キャリブレーション
def calculate_pixels_per_mm(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    xlist = []
    ylist = []

    def click_pos(event, x, y, flags, params):
        if event == cv2.EVENT_LBUTTONDOWN:
            img2 = np.copy(img)
            cv2.circle(img2, center=(x, y), radius=5, color=255, thickness=-1)
            pos_str = '(x,y)=(' + str(x) + ',' + str(y) + ')'
            font_scale = 0.8
            cv2.putText(img2, pos_str, (x + 10, y + 10), cv2.FONT_HERSHEY_PLAIN, font_scale, 255, 2, cv2.LINE_AA)
            cv2.imshow('window', img2)
            xlist.append(x)
            ylist.append(y)

    cv2.imshow('window', img)
    cv2.setMouseCallback('window', click_pos)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    dis_list = []
    for a in range(len(xlist) - 1):
        x1 = xlist[a]
        x2 = xlist[a + 1]
        y1 = ylist[a]
        y2 = ylist[a + 1]
        dis = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
        dis_list.append(dis)

    pixels_per_mm = sum(dis_list) / float(len(dis_list))
    return pixels_per_mm

image_path = 'C:\\Users\\flow\\Desktop\\calibration.jpg'
pixels_per_mm = calculate_pixels_per_mm(image_path)
print('1mmあたりのピクセル数:',pixels_per_mm )



1mmあたりのピクセル数: 68.3430541715335


In [59]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import math
import os
import pandas as pd
# LaTeX レンダリングを有効にする
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

# ----------------------------------------------------------------
# 解析法
def get_frame_rate(video_path):
    """
    ビデオファイルからフレームレートを取得する。

    Args:
    video_path: ビデオファイルのパス。

    Returns:
    フレームレート（秒あたりのフレーム数）。
    """
    # ビデオを読み込む
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("ビデオファイルを開けませんでした。")

    # フレームレートを取得
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return frame_rate


def select_roi(frame):
    """
    ユーザーによる手動でのROI選択。

    Args:
    frame: ROIを選択するための画像フレーム。

    Returns:
    選択されたROIの座標のタプル。
    """
    roi_front = cv2.selectROI("Select front Bubble", frame, False, False)
    roi_side = cv2.selectROI("Select side Bubble", frame, False, False)
    cv2.destroyAllWindows()
    return roi_front, roi_side

def process_video(video_path, roi_front, roi_side, start_frame=13, end_frame=890):
    """
    開始フレームから終了フレームまでの間のフレームを処理し、選択されたROIを使用して動画をクロップする。

    Args:
    video_path: 処理する動画ファイルのパス。
    roi_front: 側面の気泡のROI。
    roi_side: 正面の気泡のROI。
    start_frame: 処理を開始するフレームの番号（オプション、デフォルトは0）。
    end_frame: 処理を終了するフレームの番号（オプション、デフォルトはNoneで、最後まで処理）。

    Returns:
    処理されたフレームのリスト。
    """
    # 動画を読み込む
    cap = cv2.VideoCapture(video_path)
    processed_frames = []

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count < start_frame:
            frame_count += 1
            continue
        
        if end_frame is not None and frame_count > end_frame:
            break
        
        # 同じROIを使用して各フレームをクロップする
        front_x, front_y, front_w, front_h = roi_front
        side_x, side_y, side_w, side_h = roi_side
        cropped_front = frame[front_y:front_y+front_h, front_x:front_x+front_w]
        cropped_side = frame[side_y:side_y+side_h, side_x:side_x+side_w]
        
        # 処理されたフレームをリストに追加する
        processed_frames.append((cropped_front, cropped_side))

        frame_count += 1

    # すべてのウィンドウを閉じる
    cap.release()
    return processed_frames

def show_processed_frames(processed_frames):
    for i, (cropped_front, cropped_side) in enumerate(processed_frames):
        plt.figure(figsize=(12, 6))  # 図のサイズを設定
        # 側面の気泡画像を表示
        plt.subplot(1, 2, 1)  # 1行2列のグリッドの最初の位置にプロット
        plt.imshow(cv2.cvtColor(cropped_front, cv2.COLOR_BGR2RGB))
        plt.title(f'front Bubble Frame {i}')

        # 正面の気泡画像を表示
        plt.subplot(1, 2, 2)  # 1行2列のグリッドの2番目の位置にプロット
        plt.imshow(cv2.cvtColor(cropped_side, cv2.COLOR_BGR2RGB))
        plt.title(f'side Bubble Frame {i}')

        plt.show()


def visualize_contours_with_area(image, contours, image_type):
    """
    画像に輪郭とその面積を描画し、コンソールに輪郭の情報を出力します。

    Args:
    image: 輪郭を描画する画像。
    contours: 輪郭のリスト。
    image_type: 画像のタイプ（'front' または 'Front'）。

    Returns:
    輪郭が描画された画像。
    """
    # 面積計算とソート
    contours_with_area = [(cv2.contourArea(c), c) for c in contours]
    contours_with_area.sort(reverse=True, key=lambda x: x[0])
    filtered_contours = []
    # 番号付けと描画、そしてコンソールへの出力
    for i, (area, contour) in enumerate(contours_with_area):
        if i < 3:  # 最大面積の輪郭から3個を表示
            # 輪郭のバウンディングボックスを取得
            x, y, w, h = cv2.boundingRect(contour)
            cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
            
            # テキストの描画位置（バウンディングボックスの上部）
            text_position = (x+50, y+15)
            text = f"{i}: {area}"
            cv2.putText(image, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            # print(f"{image_type} Image - Contour #{i}: Area = {area}, Bounding Box = ({x}, {y}, {w}, {h})")
            filtered_contours.append(contour)
            
    return image,filtered_contours

# 円形度出力
def calculate_circularity(contour):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return 0
    circularity = 4 * np.pi * (area / (perimeter ** 2))
    return circularity

def determine_max_circularity(contours, image_type):
    """
    与えられた輪郭の中で最も円形度が高いものを選択し、その円形度を出力する。

    Args:
    contours: 輪郭のリスト。
    image_type: 輪郭のタイプ（'front' または 'Front'）。

    Returns:
    最も円形度が高い輪郭とその円形度。
    """
    # 各輪郭の円形度を計算
    circularity_contours = [(calculate_circularity(c), c) for c in contours]

    # 円形度のリストのみを抽出
    circularities = [c[0] for c in circularity_contours]

    # 各輪郭の円形度とタイプを出力
    # for i, circularity in enumerate(circularities):
        # print(f"{image_type} Image - Contour #{i}: Circularity = {circularity:.2f}")

    # 円形度が最大の輪郭を見つける
    max_circularity, max_contour = max(circularity_contours, key=lambda x: x[0])

    # 最大の円形度を持つ輪郭の情報を出力
    # print(f"{image_type} Image - Max Circularity: {max_circularity:.2f}")

    return max_contour, max_circularity

def visualize_max_circularity_contour(image, contour, circularity, image_type):
    """
    画像に最も円形度が高い輪郭とその面積、円形度を描画します。

    Args:
    image: 輪郭を描画する画像。
    contour: 最も円形度が高い輪郭。
    circularity: 輪郭の円形度。
    image_type: 画像のタイプ（'front' または 'Front'）。

    Returns:
    更新された画像。
    """
    # 輪郭の面積を計算
    area = cv2.contourArea(contour)

    # 輪郭のバウンディングボックスを取得
    x, y, w, h = cv2.boundingRect(contour)

    # 輪郭を画像に描画
    cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)

    # 面積と円形度の情報をテキストとして描画
    text1 = f"Area: {area:.2f}"
    text2 = f"Circularity: {circularity:.2f}"
    cv2.putText(image, text1, (x, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    cv2.putText(image, text2, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    # 輪郭の情報を出力
    # print(f"{image_type} Image - Contour Area: {area:.2f}, Circularity: {circularity:.2f}")

    return image


def show_step_images(front_image, side_image, step_title):
    """
    画像処理の各ステップの結果を1行2列のグリッドに表示する関数です。

    Args:
    front_image: 正面の気泡画像。
    side_image: 側面の気泡画像。
    step_title: 現在の処理ステップのタイトル。
    """
    plt.figure(figsize=(12, 6))  # 図のサイズを設定

    # 側面の気泡画像を1行2列のグリッドの最初の位置にプロット
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(front_image, cv2.COLOR_BGR2RGB))
    plt.title(f'front {step_title}')

    # 正面の気泡画像を1行2列のグリッドの2番目の位置にプロット
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(side_image, cv2.COLOR_BGR2RGB))
    plt.title(f'side {step_title}')

    plt.show()

# 画像保存する
def create_and_save_images(front_gray, side_gray, output_base_dir, step_name, file_name):
    # ステップごとのディレクトリを作成
    step_dir = os.path.join(output_base_dir, step_name)
    if not os.path.exists(step_dir):
        os.makedirs(step_dir)

    # front_gray と side_gray の高さを統一する
    desired_height = front_gray.shape[0]  # front_gray の高さを基準にする
    side_gray_resized = cv2.resize(side_gray, (side_gray.shape[1], desired_height))

    # front_gray と side_gray を横に連結
    concatenated_image = np.hstack((front_gray, side_gray_resized))

    # 画像を保存
    file_path = os.path.join(step_dir, file_name)
    cv2.imwrite(file_path, concatenated_image)

# 各画像前処理関数化
def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def apply_gaussian_blur(image):
    return cv2.GaussianBlur(image, (1, 1), 0)

def apply_threshold(image):
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary_image

def find_contours(image):
    contours, _ = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    return contours

def draw_contours(original_image, contours):
    return cv2.drawContours(original_image.copy(), contours, -1, (0, 255, 0), 2)

# シャープニングフィルタ関数
def further_adjust_sharpening(image, core_strength=8.0, surround_strength=-0.05):
    # カーネルの中心値(core_strength)と周囲の値(surround_strength)を調整
    sharpening_kernel = np.array([
        [surround_strength, surround_strength, surround_strength],
        [surround_strength, core_strength, surround_strength],
        [surround_strength, surround_strength, surround_strength]
    ])
    # カーネルの和が1になるように正規化
    sharpening_kernel /= np.sum(sharpening_kernel) if np.sum(sharpening_kernel) != 0 else 1
    
    # シャープニングフィルタを適用
    sharpened_image = cv2.filter2D(image, -1, sharpening_kernel)
    return sharpened_image

# 画像前処理
def process_and_visualize_steps(processed_frames, output_base_dir, visualize_steps=True):
    """
    動画の各フレームに対して画像処理を行い、必要に応じて処理のステップを可視化します。

    Args:
    processed_frames: 処理するフレームのリスト（側面と正面の画像のペア）。
    output_dir: 画像を保存するベースディレクトリ。
    visualize_steps: 処理のステップを可視化するかどうか。

    Returns:
    処理されたフレームのリストと各ステップの画像。
    """
    
    
    processed_images = []
    side_contours_list = []  # 側面の気泡の輪郭リスト
    front_contours_list = []  # 正面の気泡の輪郭リスト

    for i, (cropped_front, cropped_side) in enumerate(processed_frames):
        steps_images = {}
        # 元画像保存
        original_front_frames = cropped_front
        original_side_frames = cropped_side
        step_name = "original"
        file_name = f'original{i}.bmp'
        create_and_save_images(original_front_frames, original_side_frames, output_base_dir, step_name, file_name)
        
        # グレースケール化
        front_gray = convert_to_grayscale(cropped_front)
        side_gray = convert_to_grayscale(cropped_side)
        step_name = "gray"
        # ファイル名を生成
        file_name = f'gray{i}.bmp'
        create_and_save_images(front_gray, side_gray, output_base_dir, step_name, file_name)
        
        # シャープニングフィルタを適用
        front_sharpened = further_adjust_sharpening(front_gray)
        side_sharpened = further_adjust_sharpening(side_gray)
        step_name = "sharpened"
        file_name = f'sharpened{i}.bmp'
        create_and_save_images(front_sharpened, side_sharpened, output_base_dir, step_name, file_name)

        
        # 二値化
        front_binary = apply_threshold(front_sharpened)
        side_binary = apply_threshold(side_sharpened)
        step_name = "binary"
        file_name = f'binary{i}.bmp'
        create_and_save_images(front_binary,side_binary, output_base_dir, step_name, file_name)

        # 輪郭検出
        front_contours = find_contours(front_binary)
        side_contours = find_contours(side_binary)
        
        # 輪郭を描画
        front_contour_image = draw_contours(cropped_front.copy(), front_contours)
        side_contour_image = draw_contours(cropped_side.copy(), side_contours)
        step_name = "contour"
        file_name = f'contour{i}.bmp'
        create_and_save_images(front_contour_image, side_contour_image, output_base_dir, step_name, file_name)
        
        # 側面と正面の画像に対して面積フィルタリング関数を適用
        front_processed_image, front_filtered_contours = visualize_contours_with_area(cropped_front.copy(), front_contours, "front")
        side_processed_image, side_filtered_contours = visualize_contours_with_area(cropped_side.copy(), side_contours, "side")
        step_name = "contours_with_area"
        file_name = f'contours_with_area{i}.bmp'
        create_and_save_images(front_processed_image,side_processed_image, output_base_dir, step_name, file_name)
        
        
        # 最も円形度が高い輪郭を特定
        max_front_contour, max_circularity_front = determine_max_circularity(front_filtered_contours, "front")
        max_side_contour, max_circularity_side = determine_max_circularity(side_filtered_contours, "side")
        
        
        # 最も円形度が高い輪郭とその面積、円形度を描画
        front_final_image = visualize_max_circularity_contour(cropped_front.copy(), max_front_contour, max_circularity_front, "front")
        side_final_image = visualize_max_circularity_contour(cropped_side.copy(), max_side_contour, max_circularity_side, "side")
        step_name = "max_circularity_contour"
        file_name = f'max_circularity_contour{i}.bmp'
        create_and_save_images(front_final_image, side_final_image, output_base_dir, step_name, file_name)
        
        
        # 最も円形度が高い輪郭をリストに追加
        front_contours_list.append(max_front_contour)
        side_contours_list.append(max_side_contour)
        
        # 可視化
        # if visualize_steps:
        #     show_step_images(front_gray, side_gray, f'Grayscale Frame {i}')
        #     show_step_images(front_blurred, side_blurred, f'Blurred Frame {i}')
        #     show_step_images(front_binary,side_binary, f'Binary Frame {i}')
        #     show_step_images(front_contour_image, side_contour_image, f'Contours Frame {i}')
        #     show_step_images(front_processed_image, side_processed_image, f'Contours Frame with Area Frame {i}')
        #     show_step_images(front_final_image, side_final_image, f'contours_with_circularity {i}')

        # 結果を辞書に格納
        steps_images['Grayscale'] = (front_gray, side_gray)
        # steps_images['Blurred'] = (front_blurred, side_blurred)
        steps_images['Binary'] = (front_binary, side_binary)
        steps_images['Contours'] = (front_contour_image, side_contour_image)
        steps_images['Contours Frame with Area Frame'] = (front_processed_image, side_processed_image)
        steps_images['contours_with_circularity'] = ( front_final_image, side_final_image)
        processed_images.append(steps_images)

    return processed_images, front_contours_list, side_contours_list


# これ以降データ解析系の処理
# １ｍｍあたりのピクセル数　pixels_per_mm

def calculate_areas(contours):
    """
    複数の輪郭の面積を計算する。

    Args:
    contours: 輪郭データのリスト。

    Returns:
    各輪郭の面積のリスト。
    """
    areas = []
    for contour in contours:
        area_pixels = cv2.contourArea(contour)
        area_mm2 = area_pixels/pixels_per_mm # ピクセルからミリメートルに変換
        areas.append(area_mm2)
    return areas

def calculate_perimeters(contours):
    """
    複数の輪郭の周囲長を計算する。

    Args:
    contours: 輪郭データのリスト。

    Returns:
    各輪郭の周囲長のリスト。
    """
    perimeters = []
    for contour in contours:
        perimeter_pixels = cv2.arcLength(contour, True)  # ピクセル単位の周囲長
        perimeter_mm = perimeter_pixels/pixels_per_mm  # ピクセルからミリメートルに変換
        perimeters.append(perimeter_mm)
    return perimeters


def calculate_circularity_list(contours):
    """
    複数の輪郭の円形度を計算する。

    Args:
    contours: 輪郭データのリスト。

    Returns:
    各輪郭の円形度のリスト。
    """
    circularities = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            circularity = 0
        else:
            circularity = 4 * np.pi * (area / (perimeter ** 2))
        circularities.append(circularity)
    return circularities

# 気泡の長径と短径
def calculate_ellipsediameters(contours):
    """
    複数の輪郭データから気泡の長径と短径を計算する。

    Args:
    contours: 輪郭データのリスト。

    Returns:
    各輪郭の長径と短径のタプルのリスト。
    """
    diameters = []
    for contour in contours:
        _, _, w, h = cv2.boundingRect(contour)
        long_diameter = max(w, h)
        short_diameter = min(w, h)
        long_mm = long_diameter/pixels_per_mm  # ピクセルからミリメートルに変換
        short_mm = short_diameter/pixels_per_mm
        diameters.append(( long_mm, short_mm))
    return diameters

# 気泡直径の計測
def calculate_diameters(contours):
    """
    複数の輪郭データから気泡の直径を計算する。

    Args:
    contours: 輪郭データのリスト。

    Returns:
    各輪郭の直径のリスト。
    """
    diameters_list = []
    for contour in contours:
        _, _, w, h = cv2.boundingRect(contour)
        diameter_pixels = max(w, h)  # 気泡が完全な円形でない場合、最大の寸法を直径として使用
        diameter_mm = diameter_pixels/pixels_per_mm
        diameters_list.append(diameter_mm)
    return diameters_list

def calculate_single_aspect_ratio(contour):
    """
    単一の輪郭に対するアスペクト比を計算する。

    Args:
    contour: 単一の輪郭データ。

    Returns:
    計算されたアスペクト比。
    """
    # 輪郭からバウンディングボックスを計算する
    x, y, w, h = cv2.boundingRect(contour)
    # バウンディングボックスの幅と高さからアスペクト比を計算する
    aspect_ratio = float(w) / h
    return aspect_ratio

# 球の定義の閾値を決めるため
def calculate_aspect_statistics(contours):
    aspects = [calculate_single_aspect_ratio(contour) for contour in contours]
    min_aspecty = min(aspects)
    max_aspecty = max(aspects)
    avg_aspecty = sum(aspects) / len(aspects)
    return min_aspecty, max_aspecty, avg_aspecty

# 各気泡の体積
def calculate_bubble_volumes(contours, aspect_ratio_threshold=0.95):
    """
    複数の輪郭に基づいて各気泡の体積を計算する。

    Args:
    contours: 輪郭データのリスト。
    aspect_ratio_threshold: 真球と楕円を区別するためのアスペクト比の閾値。

    Returns:
    各輪郭の体積のリストと半径のリスト。
    """
    volumes = []
    radii = []
    for contour in contours:
        aspect_ratio = calculate_single_aspect_ratio(contour)
        if aspect_ratio < aspect_ratio_threshold:
            # アスペクト比が閾値より小さい場合、より円形に近いと見なし、球体の体積計算を使用
            volume, radius = calculate_volume_sphere(contour)
        else:
            # アスペクト比が閾値より大きい場合、楕円形と見なし、楕円の体積計算を使用
            volume, radius = calculate_volume_ellipse(contour)
        volumes.append(volume)
        radii.append(radius)
    return volumes, radii
    
def calculate_volume_sphere(contour):
    """
    球形気泡の体積を計算する。

    Args:
    contour: 輪郭データ。

    Returns:
    体積。
    """
    _, _, w, h = cv2.boundingRect(contour)
    diameter_pixels = max(w, h)
    radius_pixels = diameter_pixels / 2
    radius_mm = radius_pixels/pixels_per_mm
    volume = (4/3) * np.pi * (radius_mm ** 3)
    return volume, radius_mm

def calculate_volume_ellipse(contour):
    """
    楕円形気泡の体積を計算する。

    Args:
    contour: 輪郭データ。

    Returns:
    体積。
    """
    _, _, w, h = cv2.boundingRect(contour)
    a_pixels = w / 2
    b_pixels = h / 2
    a_mm = a_pixels/pixels_per_mm
    b_mm = b_pixels/pixels_per_mm
    volume = (4/3) * np.pi * a_mm * b_mm * min(a_mm, b_mm)
    # 求めた体積を球の体積と仮定して、対応する半径を算出
    radius_mm = ((3 * volume) / (4 * np.pi)) ** (1/3)
    return volume, radius_mm

# 輪郭の重心
def calculate_centroids(contours):
    """
    複数の輪郭の重心を計算する。

    Args:
    contours: 輪郭データのリスト。

    Returns:
    各輪郭の重心座標のリスト。
    """
    centroids = []
    for contour in contours:
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = 0, 0
        centroids.append((cx, cy))
    return centroids

def draw_centroids(centroids, image):
    """
    画像上に重心座標を描画する。

    Args:
    centroids: 複数の輪郭の重心座標のリスト。
    image: 画像データ。

    Returns:
    重心が描画された画像。
    """
    for centroid in centroids:  # 各重心座標はタプル (cx, cy) として格納されています
        cx, cy = centroid
        cv2.circle(image, (cx, cy), 2, (255, 0, 0), -1)  # 重心座標に青い円を描画
    return image

# 各フレーム間での気泡の上昇速度
def calculate_bubble_rise_speed(centroid_list, frame_rate):
    """
    各フレーム間での気泡の上昇速度を計算する。

    Args:
    centroid_list: 各フレームの重心座標のリスト。
    frame_rate: ビデオのフレームレート。

    Returns:
    各フレーム間の上昇速度のリスト（ピクセル/秒）。
    """
    rise_speeds = []
    for i in range(len(centroid_list) - 1):
        x1, y1 = centroid_list[i]
        x2, y2 = centroid_list[i + 1]
        # 2点間の距離を計算
        distance_pixels = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        # 時間間隔（秒）を計算
        time_interval = 1 / frame_rate
        # 上昇速度（ピクセル/秒）
        speed = distance_pixels / time_interval
        # 上昇速度（ミリメートル/秒）＝距離（ミリメートル） / 時間（秒）
        speed_mm_per_sec = (distance_pixels/pixels_per_mm) / time_interval
        rise_speeds.append(speed_mm_per_sec)
    return rise_speeds

# アスペクト比を計算
def calculate_aspect_ratio(contour):
    """
    輪郭のアスペクト比を計算する。

    Args:
    contour: 輪郭データ。

    Returns:
    計算されたアスペクト比のリスト。
    """
    aspect_ratios = []
    
    for c in contour:
        x, y, w, h = cv2.boundingRect(c)
        aspect_ratio = w / h
        aspect_ratios.append(aspect_ratio)
    
    return aspect_ratios

# 体積計算の指標として使えるか確認するため輪郭のバウンディングボックスを描画
def draw_bounding_box(image, contour):
    """
    画像に輪郭のバウンディングボックスを描画し、その縦横比をテキストとして描画する。

    Args:
    image: 輪郭を描画する元の画像。
    contour: 輪郭データ。

    Returns:
    バウンディングボックスと縦横比のテキストが描画された画像。
    """
    # バウンディングボックスを取得
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = w / h
    
    # バウンディングボックスを描画
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # 縦横比をテキストとして描画
    text = f"Aspect Ratio: {aspect_ratio:.2f}"
    cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    
    return image


# 時間間隔の計算
def calculate_time_interval(frame_rate):
    return 1.0 / frame_rate  # 秒単位の時間間隔


# 曲率算出
def calculate_curvature_modified(contours_list):
    all_curvatures = []

    for contour in contours_list:
        approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
        
        # extended_approx で次元を合わせるためにリシェイプする
        first_point = approx[-1].reshape(1, 1, 2)
        last_point = approx[0].reshape(1, 1, 2)
        
        # 最初の点と最後の点の曲率も計算するためにリストを拡張
        extended_approx = np.vstack((first_point, approx, last_point))
        
        curvatures = []
        for i in range(1, len(extended_approx) - 1):
            p1 = extended_approx[i - 1][0]
            p2 = extended_approx[i][0]
            p3 = extended_approx[i + 1][0]

            # 角度を計算
            angle = np.abs(np.arctan2(p3[1] - p2[1], p3[0] - p2[0]) - np.arctan2(p2[1] - p1[1], p2[0] - p1[0]))
            # 0で割り算が発生しないようにチェック
            if angle != 0:
                curvature = np.abs(1 / (2 * np.sin(angle / 2)))
            else:
                curvature = 0
            curvatures.append(curvature)
        
        all_curvatures.append(curvatures)

    return all_curvatures


# 近似された輪郭を描画する処理
def draw_numbered_contour_points_outside_improved(contour, image, font_scale=0.4, thickness=1):
    """
    画像に近似された輪郭を描画し、輪郭上の点に番号を付け、点の外側に描画する。

    Args:
    contour: 輪郭データ。
    image: 輪郭を描画する元の画像。
    font_scale: テキストのフォントスケール。
    thickness: テキストの太さ。

    Returns:
    番号付きで輪郭上の点が描画された画像。
    """
    # 曲線を多角形近似で単純化
    approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
    
    # 近似された輪郭を描画
    cv2.polylines(image, [approx], True, (255, 0, 0), 2)
    
    # 近似された輪郭の点の重心を計算
    M = cv2.moments(contour)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    
    # 近似された輪郭の点に番号を付けて描画
    for i, p in enumerate(approx, start=1):
        point = tuple(p[0])
        cv2.circle(image, point, 3, (0, 255, 0), -1)
        
        # テキストを描画する位置を決定する
        # 重心から離れる方向にテキストをオフセットする
        offset_x = -15 if point[0] < cx else 15
        offset_y = -15 if point[1] < cy else 15
        text_origin = (point[0] + offset_x, point[1] + offset_y)
        
        # テキストを描画
        cv2.putText(image, str(i), text_origin, cv2.FONT_HERSHEY_SIMPLEX, 
                    font_scale, (0, 0, 0), thickness, cv2.LINE_AA)
    
    return image


# 回転角度算出
def calculate_and_draw_ellipses(contours, image):
    """
    複数の輪郭に基づいて楕円を計算し、画像に描画する。

    Args:
    contours: 輪郭データのリスト。
    image: 描画する元の画像。

    Returns:
    描画された画像と、各楕円の中心座標、長径、短径、傾斜角のリスト。
    """
    ellipse_params = []
    for contour in contours:
        ellipse = cv2.fitEllipse(contour)
        cv2.ellipse(image, ellipse, (0, 255, 0), 2)  # 楕円を描画

        center = ellipse[0]
        axes = ellipse[1]
        rotation_angle = ellipse[2]
        long_axis = max(axes)
        short_axis = min(axes)

        ellipse_params.append((center, long_axis, short_axis, rotation_angle))

    return ellipse_params, image

# 剪断速度
def calculate_shear_rate(radii, frame_rate):
    """
    半径のリストとフレームレートから剪断速度を計算する。

    Args:
    radii: 時間に対する半径のリスト。
    frame_rate: フレームレート。

    Returns:
    剪断速度のリスト。
    """
    shear_rates = []
    for i in range(1, len(radii)):
        previous_radius = radii[i - 1]
        current_radius = radii[i]
        time_interval = 1.0 / frame_rate  # 1フレームあたりの秒数
        radius_derivative = ((current_radius / previous_radius) ** 2 - 1) / time_interval

        # 気泡表面速度を計算
        surface_velocity = (current_radius - previous_radius) / time_interval

        # 剪断速度を計算
        shear_rate = np.sqrt(2 * (radius_derivative ** 2) + 4 * ((surface_velocity / previous_radius) ** 2))
        shear_rates.append(shear_rate)
    return shear_rates

# 伸長速度
def calculate_extension_speed(perimeters, frame_rate):
    """
    複数の輪郭の周囲長とフレームレートから伸長速度を計算する。

    Args:
    perimeters: 周囲長のリスト（前のフレームから後のフレームの順）。
    frame_rate: フレームレート（1秒あたりのフレーム数）。

    Returns:
    各フレーム間の伸長速度のリスト。
    """
    extension_speeds = []
    for i in range(1, len(perimeters)):
        previous_perimeter = perimeters[i - 1]
        current_perimeter = perimeters[i]
        time_interval = calculate_time_interval(frame_rate)  # 1フレームあたりの秒数
        # 前の周囲長が0でないことを確認
        if previous_perimeter != 0:
            extension_speed = ((current_perimeter - previous_perimeter) / previous_perimeter) / time_interval
        else:
            # 前の周囲長が0の場合、伸長速度は計算できない（または0と定義する）ため、適切な値を設定する
            extension_speed = 0  # または他の適切な値や処理
        extension_speeds.append(extension_speed)
    return extension_speeds

# ディレクトリ名
# 全体；various_shapes,真球；spherical, ②球形で膨張・収縮.mp4, 回転体；_rotating_flat_ellipse, ①扁平回転.mp4, 縦扁平楕円；vertical-flat_ellipse, ③縦扁平で膨張・収縮.mp4
# 画像処理path指定
video_path = 'C:\\Users\\flow\\Desktop\\今さんデータ\\①扁平回転.mp4'
output_base_dir = 'C:\\Users\\flow\\Desktop\\master_thesis_rotating_flat_ellipse\\'
# 画像解析処理path指定
save_directory = 'C:\\Users\\flow\\Desktop\\master_thesis_rotating_flat_ellipse\\graphes\\'
save_directory2 = 'C:\\Users\\flow\\Desktop\\master_thesis_rotating_flat_ellipse\\combined_time_series_subplot_graphes\\'
save_path= 'C:\\Users\\flow\\Desktop\\master_thesis_rotating_flat_ellipse\\statistics'
# 最初のフレームでROIを選択するために動画を読み込む
cap = cv2.VideoCapture(video_path)
fps = get_frame_rate(video_path)
print(fps)
ret, first_frame = cap.read()
if not ret:
    print("Failed to read the video")
else:
    # ROIを選択する
    roi_front, roi_side = select_roi(first_frame)

    # 選択されたROIを使用して動画を処理
    processed_frames = process_video(video_path, roi_front, roi_side)
    print("Processed {} frames".format(len(processed_frames)))
    # print(processed_frames)
    # show_processed_frames(processed_frames)  
    
    processed_images, front_contours_list, side_contours_list = process_and_visualize_steps(processed_frames, output_base_dir, visualize_steps=True)
    
# 解析
# 側面の気泡についてアスペクト比の統計を計算
side_min_aspect, side_max_aspect, side_avg_aspect = calculate_aspect_statistics(side_contours_list)

# 正面の気泡についてアスペクト比の統計を計算
front_min_aspect, front_max_aspect, front_avg_aspect = calculate_aspect_statistics(front_contours_list)

# 結果を表示
print(f"Side Bubble aspects: Min = {side_min_aspect}, Max = {side_max_aspect}, Avg = {side_avg_aspect}")
print(f"Front Bubble aspects: Min = {front_min_aspect}, Max = {front_max_aspect}, Avg = {front_avg_aspect}")

# 正面の輪郭の解析
front_areas = calculate_areas(front_contours_list)
front_perimeters =calculate_perimeters(front_contours_list)
front_aspect_ratios = calculate_aspect_ratio(front_contours_list)
front_circularity = calculate_circularity_list(front_contours_list)
front_ellipsediameters = calculate_ellipsediameters(front_contours_list)
front_diameters = calculate_diameters(front_contours_list)
front_volumes, front_radii = calculate_bubble_volumes(front_contours_list, aspect_ratio_threshold=0.95)
front_centroids = calculate_centroids(front_contours_list)
front_bubble_rise_speed = calculate_bubble_rise_speed(front_centroids, fps)
front_curvatures = calculate_curvature_modified(front_contours_list)
front_shear_rates = calculate_shear_rate(front_radii, fps)
front_extension_speeds = calculate_extension_speed(front_perimeters, fps)

# 側面の輪郭の解析
side_areas = calculate_areas(side_contours_list)
side_perimeters =calculate_perimeters(side_contours_list)
side_aspect_ratios = calculate_aspect_ratio(side_contours_list)
side_circularity = calculate_circularity_list(side_contours_list)
side_ellipsediameters = calculate_ellipsediameters(side_contours_list)
side_diameters = calculate_diameters(side_contours_list)
side_volumes, side_radii = calculate_bubble_volumes(side_contours_list, aspect_ratio_threshold=0.95)
side_centroids = calculate_centroids(side_contours_list)
side_bubble_rise_speed = calculate_bubble_rise_speed(side_centroids, fps)
side_curvatures = calculate_curvature_modified(side_contours_list)
side_shear_rates = calculate_shear_rate(side_radii, fps)
side_extension_speeds = calculate_extension_speed(side_perimeters, fps)


# processed_frames内の各フレームに対して処理を行う
for i, (cropped_front, cropped_side) in enumerate(processed_frames):
    # 正面画像に対してバウンディングボックスを描画し縦横比を計算
    front_contour = front_contours_list[i]  # 正面の輪郭情報を取得
    front_image_with_box = draw_bounding_box(cropped_front.copy(), front_contour)
    front_curvatures_image = draw_numbered_contour_points_outside_improved(front_contour,cropped_front.copy())
    front_draw_centroids_image = draw_centroids(front_centroids,cropped_front.copy())
    
    # 側面画像に対してバウンディングボックスを描画し縦横比を計算
    side_contour = side_contours_list[i]  # 側面の輪郭情報を取得
    side_image_with_box = draw_bounding_box(cropped_side.copy(), side_contour)
    side_curvatures_image = draw_numbered_contour_points_outside_improved(side_contour,cropped_side.copy())
    side_draw_centroids_image = draw_centroids(side_centroids,cropped_side.copy())
    

    # 解析結果を保存
    step_name = 'bounding_box'
    file_name = f'bounding_box_{i}.bmp'
    create_and_save_images(front_image_with_box, side_image_with_box, output_base_dir, step_name, file_name)

    # 曲率処理
    step_name = 'curvatures'
    file_name = f'curvature_{i}.bmp'
    create_and_save_images(front_curvatures_image,side_curvatures_image, output_base_dir, step_name, file_name)

    step_name = 'draw_centroids'
    file_name = f'draw_centroids_{i}.bmp'
    create_and_save_images(front_draw_centroids_image, side_draw_centroids_image, output_base_dir, step_name, file_name)
    


29.97002997002997
Processed 878 frames
Side Bubble aspects: Min = 0.5853658536585366, Max = 1.0, Avg = 0.877396990531443
Front Bubble aspects: Min = 0.47435897435897434, Max = 0.9306930693069307, Avg = 0.7341721308229061


In [60]:
  
# グラフ作成
# 1つのデータセットを1つのグラフ
def plot_change_over_time(data, title, x_label, y_label, save_directory, filename):
    frame_numbers = np.arange(1, len(data) + 1)
    
    # データをプロット
    plt.plot(frame_numbers, data, marker='o', color='blue')
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.xticks(frame_numbers)
    
    # 保存ディレクトリが存在しない場合、作成
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    # ファイルを指定のディレクトリに保存
    save_path = os.path.join(save_directory, filename)
    plt.savefig(save_path)
    plt.tight_layout()
    plt.show()


# 正面と側面のデータを同じグラフに載せる関数
# 2つのデータセット（正面データと側面データ）を1つのグラフに並べて表示
def create_combined_time_series_subplots(front_data, side_data, title, x_label, y_label, save_directory, prefix, frame_interval=90, y_min=None, y_max=None):
    # データポイントの数
    num_data_points = len(front_data)
    
    # サブプロット数 (指定したフレームの間隔ごとに1つ)
    num_subplots = num_data_points // frame_interval + (1 if num_data_points % frame_interval != 0 else 0)
    
    # 保存ディレクトリが存在しない場合、作成
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    # サブプロットごとに処理
    for i in range(num_subplots):
        start_idx = i * frame_interval
        end_idx = min((i + 1) * frame_interval, num_data_points)
        
        # サブプロットごとのデータを取得
        front_data_sub = front_data[start_idx:end_idx]
        side_data_sub = side_data[start_idx:end_idx]
        
        # サブディレクトリを作成
        subdir_name = f"frames_{start_idx}-{end_idx}"
        subdir_path = os.path.join(save_directory, subdir_name)
        if not os.path.exists(subdir_path):
            os.makedirs(subdir_path)
        
        # サブプロットごとのグラフを作成
        frame_numbers = np.arange(start_idx, end_idx)
        plt.figure(figsize=(10, 6))
        plt.plot(frame_numbers, front_data_sub, label='Front', marker='o', color='blue')
        plt.plot(frame_numbers, side_data_sub, label='Side', marker='o', color='green')
        plt.title(f"{title} (Frames {start_idx}-{end_idx})", fontsize=14)
        plt.xlabel(x_label, fontsize=16)
        plt.ylabel(y_label, fontsize=16)    
        # plt.xlabel(r'$\textit{' + x_label + '}$')
        # plt.ylabel(r'$\textit{' + y_label + '}$')
        
        # y軸の範囲を設定する
        if y_min is not None or y_max is not None:
            plt.ylim(y_min, y_max)
        
        plt.xticks(np.arange(start_idx, end_idx + 1, 5))
        plt.legend()
        
        # ファイルをサブディレクトリに保存
        filename = f"{prefix}_{start_idx}-{end_idx}.png"
        save_path = os.path.join(subdir_path, filename)
        plt.savefig(save_path)
        plt.tight_layout()
        plt.close()

# 正面と側面の気泡面積の変化を同じグラフにプロット
create_combined_time_series_subplots(front_areas, side_areas, 'Bubble Area ', 'Frame Number[-]', 'Area [mm²]', save_directory2, 'Combined_Area.png', frame_interval=90, y_min=20, y_max=120)
# 正面と側面の気泡周囲長の変化を同じグラフにプロット
create_combined_time_series_subplots(front_perimeters, side_perimeters, 'Bubble Perimeter ', 'Frame Number[-]', 'Perimeter [mm]', save_directory2, 'Combined_Perimeter.png', frame_interval=90, y_min=2.0, y_max=5.0 )
# 正面と側面の気泡アスペクト比の変化を同じグラフにプロット
create_combined_time_series_subplots(front_aspect_ratios, side_aspect_ratios, 'Bubble Aspect Ratio ', 'Frame Number[-]', 'Aspect Ratio[-]', save_directory2, 'Combined_Aspect_Ratio.png', frame_interval=90, y_min=0.4, y_max=1.2)
# 正面と側面の気泡円形度の変化を同じグラフにプロット
create_combined_time_series_subplots(front_circularity, side_circularity, 'Bubble Circularity ', 'Frame Number[-]', 'Circularity[-]', save_directory2, 'Combined_Circularity.png', frame_interval=90, y_min=0.6, y_max=1.0)
# 正面と側面の気泡直径の変化を同じグラフにプロット
create_combined_time_series_subplots(front_diameters, side_diameters, 'Bubble Diameter ', 'Frame Number[-]', 'Diameter [mm]', save_directory2, 'Combined_Diameter.png', frame_interval=90, y_min=0.8, y_max=1.6)
# 正面と側面の気泡体積の変化を同じグラフにプロット
create_combined_time_series_subplots(front_volumes, side_volumes, 'Bubble Volume ', 'Frame Number[-]', 'Volume [mm³]', save_directory2, 'Combined_Volume.png', frame_interval=90, y_min=0.1, y_max=2.0)
# 正面と側面の気泡上昇速度の変化を同じグラフにプロット
create_combined_time_series_subplots(front_bubble_rise_speed, side_bubble_rise_speed, 'Bubble Rising Velocity ', 'Frame Number[-]', 'Rising Velocity [mm/s]', save_directory2, 'Combined_Rising_Velocity.png', frame_interval=90, y_min=0.0, y_max=1.0)
# 正面と側面の気泡剪断速度の変化を同じグラフにプロット
create_combined_time_series_subplots(front_shear_rates, side_shear_rates, 'Bubble Shear Rate ', 'Frame Number[-]', 'Shear Rate $\gamma$ [$s^-1$]', save_directory2, 'Combined_Shear_Rate.png', frame_interval=90, y_min=0.0, y_max=19)
# 正面と側面の気泡伸長速度の変化を同じグラフにプロット
create_combined_time_series_subplots(front_extension_speeds, side_extension_speeds, 'Bubble Extension Speed ', 'Frame Number[-]', 'Extension Speed[$s^-1$]', save_directory2, 'Combined_Extension_Speed.png', frame_interval=90, y_min=-1.8, y_max=2.0)




In [20]:
# データ数値分析
def calculate_statistics_with_frames(data, save_path=None, unit=None):
    """
    指定されたデータリストから平均値、中央値、分散、標準偏差、最小値、最小値のフレーム数、最大値、最大値のフレーム数を計算し、
    データフレームで結果を返します。

    Parameters:
    data (list of float): データ値のリスト。
    unit (str): データ値の単位（任意）。デフォルトは None。

    Returns:
    pandas.DataFrame: 平均値、中央値、分散、標準偏差、最小値、最小値のインデックス、最大値、最大値のインデックスを含むデータフレーム。
    """
    if len(data) == 0:
        raise ValueError("データリストが空です。")
    
    mean = np.mean(data)
    median = np.median(data)
    variance = np.var(data)
    std_deviation = np.std(data)
    minimum = np.min(data)
    minimum_index = np.argmin(data)  # 最小値のインデックス
    maximum = np.max(data)
    maximum_index = np.argmax(data)  # 最大値のインデックス
    
    statistics = {
        "Mean": mean,
        "Median": median,
        "Variance": variance,
        "Standard Deviation": std_deviation,
        "Minimum": minimum,
        "Minimum Index": minimum_index,
        "Maximum": maximum,
        "Maximum Index": maximum_index
    }

    # 単位が指定されている場合、データフレームに単位を追加
    if unit is not None:
        statistics["Unit"] = unit
    
    # 統計情報をデータフレームに格納
    df = pd.DataFrame.from_dict(statistics, orient="index", columns=["Value"])
    # 結果を指定されたファイルに保存
    if save_path is not None:
        df.to_csv(save_path)
    return df

# 正面の気泡の面積の統計情報を計算
front_area_statistics = calculate_statistics_with_frames(front_areas, save_path, unit="mm²")
print("Front Bubble Area Statistics:")
print(front_area_statistics) 

# 側面の気泡の面積の統計情報を計算
side_area_statistics = calculate_statistics_with_frames(side_areas, save_path, unit="mm²")
print("\nSide Bubble Area Statistics:")
print(side_area_statistics)

# 正面の気泡の平均体積の統計情報を計算
front_volume_statistics = calculate_statistics_with_frames(front_volumes, save_path, unit="mm³")
print("Front Bubble Volume Statistics:")
print(front_volume_statistics)

# 側面の気泡の平均体積の統計情報を計算
side_volume_statistics = calculate_statistics_with_frames(side_volumes, save_path, unit="mm³")
print("\nSide Bubble Volume Statistics:")
print(side_volume_statistics)

# 正面の気泡の輪郭の長さの統計情報を計算
front_perimeter_statistics = calculate_statistics_with_frames(front_perimeters, save_path, unit="mm")
print("Front Bubble Perimeter Statistics:")
print(front_perimeter_statistics)

# 側面の気泡の輪郭の長さの統計情報を計算
side_perimeter_statistics = calculate_statistics_with_frames(side_perimeters, save_path, unit="mm")
print("\nSide Bubble Perimeter Statistics:")
print(side_perimeter_statistics)

# 正面の気泡径の長さの統計情報を計算
front_diameters_statistics = calculate_statistics_with_frames(front_diameters, save_path, unit="mm")
print("Front Bubble diameter Statistics:")
print(front_diameters_statistics)

# 側面の気泡径の長さの統計情報を計算
side_diameters_statistics = calculate_statistics_with_frames(side_diameters, save_path, unit="mm")
print("\nSide Bubble diameter Statistics:")
print(side_diameters_statistics)

# 正面の気泡の平均円形度の統計情報を計算
front_circularity_statistics = calculate_statistics_with_frames(front_circularity, save_path, unit="")
print("Front Bubble Circularity Statistics:")
print(front_circularity_statistics)

# 側面の気泡の平均円形度の統計情報を計算
side_circularity_statistics = calculate_statistics_with_frames(side_circularity, save_path, unit="")
print("\nSide Bubble Circularity Statistics:")
print(side_circularity_statistics)

# 正面の気泡の平均伸長速度の統計情報を計算
front_extension_speed_statistics = calculate_statistics_with_frames(front_extension_speeds, save_path,unit="s-1")
print("Front Bubble Extension Speed Statistics:")
print(front_extension_speed_statistics)

# 側面の気泡の平均伸長速度の統計情報を計算
side_extension_speed_statistics = calculate_statistics_with_frames(side_extension_speeds, save_path, unit="s-1")
print("\nSide Bubble Extension Speed Statistics:")
print(side_extension_speed_statistics)

# 正面の気泡のせん断速度の統計情報を計算
front_shear_rates_statistics = calculate_statistics_with_frames(front_shear_rates, save_path, unit="s-1" )
print("\nFront shear_rates Statistics:")
print(front_shear_rates_statistics)
# 側面の気泡のせん断速度の統計情報を計算
side_shear_rates_statistics = calculate_statistics_with_frames(side_shear_rates, save_path, unit="s-1" )
print("\nSide shear_rates Statistics:")
print(side_shear_rates_statistics)

# 正面の気泡の上昇速度の統計情報を計算
front_bubble_rise_speed_statistics = calculate_statistics_with_frames(front_bubble_rise_speed, save_path, unit="mm/s" )
print("\nFront bubble_rise_speed Statitics:")
print(front_bubble_rise_speed_statistics)

# 側面の気泡の上昇速度の統計情報を計算
side_bubble_rise_speed_statistics = calculate_statistics_with_frames(side_bubble_rise_speed, save_path, unit="mm/s" )
print("\nSide bubble_rise_speed Statistics:")
print(side_bubble_rise_speed_statistics)

# 正面の気泡のアスペクト比の統計情報を計算
front_aspect_ratios_statistics = calculate_statistics_with_frames(front_aspect_ratios, save_path, unit="-" )
print("\nFront aspect_ratios Statistics:")
print(front_aspect_ratios_statistics)

# 側面の気泡のアスペクト比の統計情報を計算
side_aspect_ratios_statistics = calculate_statistics_with_frames(side_aspect_ratios, save_path, unit="-" )
print("\nSide aspect_ratios Statistics:")
print(side_aspect_ratios_statistics)

Front Bubble Area Statistics:
                         Value
Mean                 53.972046
Median               55.265309
Variance            109.239647
Standard Deviation   10.451777
Minimum              37.765359
Minimum Index                1
Maximum              67.117281
Maximum Index              153
Unit                       mm²

Side Bubble Area Statistics:
                        Value
Mean                55.077119
Median              56.765096
Variance            116.12495
Standard Deviation  10.776129
Minimum             37.458086
Minimum Index               0
Maximum             68.887761
Maximum Index             153
Unit                      mm²
Front Bubble Volume Statistics:
                       Value
Mean                0.562765
Median               0.57065
Variance            0.024522
Standard Deviation  0.156596
Minimum             0.325554
Minimum Index              2
Maximum             0.778394
Maximum Index             50
Unit                     mm³

Side Bu

area: 29.885992435654856
perimeter: 2.796121057526531
aspect_ratio: 0.48717948717948717
circularity: 0.7028638755411515
ellipsediameter: (1.141301057518276, 0.5560184639191601)
diameter: 1.141301057518276
volume: 0.77839364457331
radius: 0.570650528759138
centroid: (220, 156)
rise_speed: 0.0
curvature: [0.7967834811544734, 1.5661834636579792, 2.1582463699459105, 2.419441356196079, 0.6160456150672252, 2.384144198111322, 2.206022336064551, 1.6417137366497445]
shear_rate: 0.0
extension_speed: 0.0


In [62]:
# 特定のフレームの解析
# フレーム指定の曲率グラフ
def plot_curvature_points(curvatures_list, frame_index, directory, filename_prefix, y_min=0.5, y_max=3.0):
    """
    選択した輪郭の曲率を点のみでグラフにプロットする。

    Args:
    curvatures_list: 各輪郭に対する曲率のリストのリスト。
    frame_index: グラフにプロットしたい輪郭のインデックス。
    """
    # 特定のフレームの曲率を取得します。
    frame_curvatures = curvatures_list[frame_index]

    # グラフにプロットするためにインデックスリストを作成します。
    indices = list(range(1, len(frame_curvatures) + 1))

    # 曲率をプロットします（線はつながない）。
    plt.figure(figsize=(10, 4))
    plt.scatter(indices, frame_curvatures, color='blue')
    plt.title(f'Curvature points along the contour {frame_index}')
    plt.xlabel('Point Index')
    plt.ylabel('Curvature Value')
    plt.grid(True)
    
    # y軸の範囲を指定する場合
    if y_min is not None and y_max is not None:
        plt.ylim(y_min, y_max)
        
    # 保存先のパスを作成します。
    if not os.path.exists(directory):
        os.makedirs(directory)
    save_path = os.path.join(directory, f"{filename_prefix}_index{frame_index}.png")
    
    plt.savefig(save_path)  # グラフをファイルに保存
    plt.close()  # グラフを閉じる

def get_frame_data(index, areas, perimeters, aspect_ratios, circularity, ellipsediameters, diameters, volumes, radii, centroids, rise_speed, curvatures, shear_rates, extension_speeds):
    """
    指定されたインデックス番号に対応するすべてのパラメータのデータを返す。

    Args:
    index: データを取得したいフレームのインデックス。
    その他の引数: 各種計算されたパラメータのリスト。

    Returns:
    指定されたインデックスにおける各パラメータのデータが含まれる辞書。
    """
    return {
        'area': areas[index],
        'perimeter': perimeters[index],
        'aspect_ratio': aspect_ratios[index],
        'circularity': circularity[index],
        'ellipsediameter': ellipsediameters[index],
        'diameter': diameters[index],
        'volume': volumes[index],
        'radius': radii[index],
        'centroid': centroids[index],
        'rise_speed': rise_speed[index],
        'curvature': curvatures[index],
        'shear_rate': shear_rates[index],
        'extension_speed': extension_speeds[index],
    }
    
# 使用例
front_data = get_frame_data(
    101,  # 例として6番目のフレームのデータを取得
    front_areas,
    front_perimeters,
    front_aspect_ratios,
    front_circularity,
    front_ellipsediameters,
    front_diameters,
    front_volumes,
    front_radii,
    front_centroids,
    front_bubble_rise_speed,
    front_curvatures,
    front_shear_rates,
    front_extension_speeds
)

# 出力例
for key, value in front_data.items():
    print(f"{key}: {value}")
    
directory= 'C:\\Users\\flow\\Desktop\\master_thesis_rotating_flat_ellipse\\curvature_plot'
filename_prefix = 'curvature_plot'
plot_curvature_points(front_curvatures,101, directory, filename_prefix)

# def display_frame_data_as_table(frame_data):
#     """
#     特定のフレームの解析データをテーブル形式で表示する関数。

#     Args:
#     frame_data: 特定のフレームの解析データが格納された辞書。

#     Returns:
#     DataFrame形式のテーブル。
#     """
#     # データをDataFrameに変換
#     df = pd.DataFrame(frame_data.items(), columns=['Property', 'Value'])
#     # Index列を追加（特定のフレームにはインデックスが不要なので'N/A'を使用）
#     df['Index'] = 'N/A'
#     return df

# frame_data_table = display_frame_data_as_table(front_data)

# # テーブルを表示
# print(frame_data_table)

area: 29.885992435654856
perimeter: 2.796121057526531
aspect_ratio: 0.48717948717948717
circularity: 0.7028638755411515
ellipsediameter: (1.141301057518276, 0.5560184639191601)
diameter: 1.141301057518276
volume: 0.77839364457331
radius: 0.570650528759138
centroid: (220, 156)
rise_speed: 0.0
curvature: [0.7967834811544734, 1.5661834636579792, 2.1582463699459105, 2.419441356196079, 0.6160456150672252, 2.384144198111322, 2.206022336064551, 1.6417137366497445]
shear_rate: 0.0
extension_speed: 0.0


In [47]:
side_data = get_frame_data(
    404,  # 例として6番目のフレームのデータを取得
    side_areas,
    side_perimeters,
    side_aspect_ratios,
    side_circularity,
    side_ellipsediameters,
    side_diameters,
    side_volumes,
    side_radii,
    side_centroids,
    side_bubble_rise_speed,
    side_curvatures,
    side_shear_rates,
    side_extension_speeds
)

for key, value in side_data.items():
    print(f"{key}: {value}")
    
    
plot_curvature_points(side_curvatures,408, directory, filename_prefix)

area: 63.078831525144714
perimeter: 3.64746968740414
aspect_ratio: 0.8765432098765432
circularity: 0.871797738655119
ellipsediameter: (1.1851972520382097, 1.0388766036384307)
diameter: 1.1851972520382097
volume: 0.8717069928388972
radius: 0.5925986260191048
centroid: (204, 153)
rise_speed: 0.0
curvature: [1.2966006759520028, 1.3539244761697515, 1.2833533149683696, 1.3803948565192048, 1.099729769497507, 1.3238711843044304, 1.4314186086322538, 1.346291201783626]
shear_rate: 0.0
extension_speed: -0.3400525915398407
